# Browser session

> Python module to set up and manage multiple browser sessions in web application.

In [ ]:
#| default_exp application.session

In [ ]:
#| hide
from nbdev.showdoc import *

Concurrency should be taken into consideration in developing our web application. Concurrency refers to the ability of the application to handle multiple tasks or connections simultaneously, without blocking or waiting for other tasks to complete. This is crucial for ensuring that the service remains responsive and scalable, especially when serving a large number of users.

However, with our current framework, there are multiple functions that utilize global variables, which will be shared from the server-side. This can cause significant issues in a concurrent environment, such as race conditions, where multiple tasks or connections attempt to read and modify the same variable simultaneously. This may lead to unpredictable behavior, data corruption, or incorrect results. For instance, if two users' requests depend on the same global variable being updated, the updates might overwrite each other or produce inconsistent states, affecting the reliability of the application.

Our current solution to this issue involves introducing the concept of `session`. Each `session` represents one connection with a user browser and all session-specific variables are stored in a higher-level mapping of session ID to values. Our frameworks will also configured towards the use of session.

In [ ]:
#| export
import uuid

from typing import Callable, Optional
from llmcam.core.fc import *
from llmcam.core.fn_to_schema import *
from llmcam.vision.ytlive import *
from llmcam.vision.gpt4v import *
from llmcam.vision.yolo import *
from llmcam.vision.dtcam import *
from llmcam.vision.plotting import *
from llmcam.utils.store import *
from llmcam.utils.file_manager import *
from llmcam.utils.notification import *
from llmcam.utils.bash_command import *

## Session messages

As a chatbot application, we usually require the whole conversation being pasted into chat history rather than using a single new message for a smooth user experience. For simplication purpose, we introduce a mock database for storing our `messages` list for each session:

In [ ]:
#| export
session_messages = {}  # Mappping of session_id to messages

In [ ]:
#| export
def retrieve_session_message(session_id: str) -> list:
    """Retrieve the messages for a given session_id"""
    return session_messages.get(session_id, [])

In [ ]:
show_doc(retrieve_session_message)

---

### retrieve_session_message

>      retrieve_session_message (session_id:str)

*Retrieve the messages for a given session_id*

## Session tools

One major utility in our framework that uses global variables is the `llmcam.utils.store` module which updates the global tools list. To configure this towards `session` framework, we can save the `tools` instance (Python pointer) of each `session` and define a custom fixup function that retrieves this instance and passes it to the actual manager tools. The schema of these manager tools also require a new `metadata` field called `session_id` for the fixup function to identify the correct instance.

In [ ]:
#| export
session_tools = {}  # Mappping of session_id to tools

In [ ]:
#| export
def retrieve_session_tools(session_id: str) -> list:
    """Retrieve the tools for a given session_id"""
    return session_tools.get(session_id, [])

In [ ]:
show_doc(retrieve_session_tools)

---

### retrieve_session_tools

>      retrieve_session_tools (session_id:str)

*Retrieve the tools for a given session_id*

In [ ]:
#| export
# Utility functions to manage tools per session
def prepare_handler_schemas(
    session_id: str,  # Session ID to use
    fixup: Callable = None,  # Optional function to fix up the execution
):
    return [
        handler_schema(function, service_name="toolbox_manager", fixup=fixup, session_id=session_id) for \
        function in [add_api_tools, add_function_tools, remove_tools]
    ]

def execute_handler(
    function_name: str,  # Name of the function to execute
    session_id: str,  # Session ID to use
    **kwargs,  # Additional arguments to pass to the function
):
    tools = session_tools[session_id]
    return execute_handler_core(tools, function_name, **kwargs)

In [ ]:
show_doc(prepare_handler_schemas)

---

[source](https://github.com/ninjalabo/llmcam/blob/main/llmcam/application/chat_ui.py#L56){target="_blank" style="float:right; font-size:smaller"}

### prepare_handler_schemas

>      prepare_handler_schemas (session_id:str, fixup:Callable=None)

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| session_id | str |  | Session ID to use |
| fixup | Callable | None | Optional function to fix up the execution |

In [ ]:
show_doc(execute_handler)

---

[source](https://github.com/ninjalabo/llmcam/blob/main/llmcam/application/chat_ui.py#L65){target="_blank" style="float:right; font-size:smaller"}

### execute_handler

>      execute_handler (function_name:str, session_id:str, **kwargs)

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| function_name | str | Name of the function to execute |
| session_id | str | Session ID to use |
| kwargs |  |  |

## Session notifications

The notifications system is quite more complicated. It relies on also subtools to pass in sub GPT in the separate thread running notification stream. These subtools include a tool to send notification and a tool to stop the stream. The default utilities function for this uses a common global pointer `stream_thread`, which will not work in this case. Hence, we need to implement the new utilities function:

- `execute_send_notification`: Instead of implementing the actual function, a `fixup` is implemented to capitalize on the metadata field `session_id`. This function also utilizes the feature that a websocket sender function can be saved in global collections -  [example with real time Chat App](https://docs.fastht.ml/explains/websockets.html#real-time-chat-app). Therefore, it simply retrieves the sender function with `session_id` and uses it to send the message.  
- `execute_stopper`: Instead of implementing the actual function, a `fixup` is implemented to capitalize on the metadata field `session_id` and `noti_id`. The idea is that each notification stream is given a unique ID and saved in a mapping of IDs to notification streams. This mapping is also retrievable by `session_id`.

We also need to implement a custom `start_notification_stream` function which will be used as a tool by GPT Function Calling. This function will utilize `session_id` and create a unique `noti_id` to define schemas for subtools with these values as metadata. It also defines these schemas such that the `module` metadata is missing to ensure the attached `fixup` function will be called instead. However, because it uses a metadata field `session_id`, it will also need to have a `fixup` function.

In [ ]:
#| export
session_notis = {}  # Mappping of session_id to notification sender \
                    # and notification streams

In [ ]:
#| export
def retrieve_session_notis(session_id: str) -> tuple:
    """Retrieve the notification sender and streams for a given session_id"""
    return session_notis.get(session_id, None)

In [ ]:
#| export
def set_noti_sender(session_id: str, noti_sender: Callable):
    """Set the notification sender for a given session_id"""
    session_notis[session_id] = (noti_sender, {})

In [ ]:
#| export
# Utility functions to manage notifications per session
def execute_send_notification(function_name, session_id, msg, **kwargs):
    """Fixup function to send a notification."""
    global session_notis
    sender, _ = session_notis[session_id]  # Get the sender
    sender(msg)
    return 'Notification sent'

def execute_stopper(function_name, session_id, noti_id, **kwargs):
    """Fixup function to stop a notification stream."""
    global session_notis
    _, notis = session_notis[session_id]  # Get the notification streams 
    notis[noti_id].stop()  # Stop the stream with the given ID
    return 'Notification stream stopped'

# Utility to define the schemas for the sender and stopper
def prepare_sender_schema(session_id: str):
    return {
        'type': 'function',
        'function': {
            'name': 'send_notification',
            'description': 'Send a notification with a message',
            'metadata': {
                'session_id': session_id
            },
            'parameters': {'type': 'object',
                'properties': {'msg': {'type': 'string',
                'description': 'Notification message to send'}},
                'required': ['msg']},
            'fixup': f"{execute_send_notification.__module__}.{execute_send_notification.__name__}"
        }
    }

def prepare_stopper_schema(session_id: str):
    return {
        'type': 'function',
        'function': {
            'name': 'stop_notification',
            'description': 'Stop the notification stream',
            'parameters': {
                'type': 'object', 
                'properties': {
                    'noti_id': {
                        'type': 'string', 
                        'description': 'Unique UUID of the notification stream to stop, provided when the stream was started'}
                }, 
                'required': ['noti_id']},
            'metadata': {
                'session_id': session_id,
            },
            'fixup': f"{execute_stopper.__module__}.{execute_stopper.__name__}"
        }
    }

In [ ]:
show_doc(execute_send_notification)

---

[source](https://github.com/ninjalabo/llmcam/blob/main/llmcam/application/chat_ui.py#L75){target="_blank" style="float:right; font-size:smaller"}

### execute_send_notification

>      execute_send_notification (function_name, session_id, msg, **kwargs)

*Fixup function to send a notification.*

In [ ]:
show_doc(execute_stopper)

---

[source](https://github.com/ninjalabo/llmcam/blob/main/llmcam/application/chat_ui.py#L82){target="_blank" style="float:right; font-size:smaller"}

### execute_stopper

>      execute_stopper (function_name, session_id, noti_id, **kwargs)

*Fixup function to stop a notification stream.*

Implementation of custom `start_notification_stream`:

In [ ]:
#| export
def start_notification_stream(
    session_id: str,  # Session ID to use
    messages: list,  # All the previous messages in the conversation
):
    """Start a notification stream to monitor a process described in messages."""
    global session_notis
    global session_tools

    _, notis = session_notis[session_id]  # Get the notification streams

    # Define a new notification stream with a unique ID
    noti_id = str(uuid.uuid4())
    
    # Describe the sender and stopper functions
    sender_schema = prepare_sender_schema(session_id)
    stopper_schema = prepare_stopper_schema(session_id)

    # Define a function to start the stream
    def stream_starter(tools, messages):
        notis[noti_id] = StreamThread(noti_id, tools, messages)
        notis[noti_id].start()

    # Extract the tools for the session
    tools = session_tools[session_id]
    # Remove the stop_notification tool from the list of tools to avoid duplication
    tools = [ tool for tool in tools if tool['function']['name'] != 'stop_notification' ] 

    submessages = [ message for message in messages ]
    submessages.append(form_msg(
        'system',
        f'Notification stream started with ID {noti_id}. Complete the stream here.'
    ))

    # Start the notification stream
    notification_stream_core(
        tools, 
        submessages,
        stream_starter=stream_starter,
        send_notification_schema=sender_schema,
        stream_stopper_schema=stopper_schema
    )

    # Return the ID of the notification stream  
    return f"Notification stream started with ID {noti_id}" 

Define the `fixup` function for starting a notification stream which passes in `session_id` to `start_notification_stream`. Define also a function to attach `session_id` as metadata for the notification FC schema and attach the `fixup` function.

In [ ]:
#| export
def prepare_notification_schemas(
        session_id: str,  # Session ID to use
        fixup: Callable = None,  # Optional function to fix up the execution
    ):  # Prepare the notification schema
    schema = process_notification_schema(start_notification_stream)  # Get the schema for starting notification stream
    # Set additional metadata
    schema['function']['metadata']['session_id'] = session_id  
    if fixup: schema['function']['fixup'] = f"{fixup.__module__}.{fixup.__name__}"
    return schema

def execute_start_notification_stream(function_name, session_id, messages, **kwargs):
    """Fixup function to start a notification stream."""
    return start_notification_stream(session_id, messages)

## Setup

Implement some utilities to manage all session-related data:

In [ ]:
#| export
# Set up default tools from our `llmcam.utils` and `llmcam.vision` modules.
default_tools = [function_schema(fn) for fn in (
    capture_youtube_live_frame, 
    ask_gpt4v_about_image_file,
    detect_objects,
    cap,
    list_image_files,
    list_detection_files,
    plot_object,
    execute_bash_command,
    camera_address_book,
)]

In [ ]:
#| export
def init_session(session_id: Optional[str] = None):
    if session_id is None or session_id not in session_messages:
        # Initialize tools in session tools and create a session ID
        session_id = str(uuid.uuid4())

        # Add default tools, prepare handler schemas, prepare notification schemas, and prepare stopper schema
        session_tools[session_id] = []
        session_tools[session_id].extend(prepare_handler_schemas(session_id, execute_handler))
        session_tools[session_id].extend(default_tools)
        session_tools[session_id].append(prepare_notification_schemas(session_id, execute_start_notification_stream))
        session_tools[session_id].append(prepare_stopper_schema(session_id))

        # Initialize messages in session messages
        session_messages[session_id] = []
    
    return session_id

In [ ]:
#| export
def remove_session(session_id: str):
    """Remove the session with the given session_id"""
    if session_id in session_messages:
        del session_messages[session_id]
    if session_id in session_tools:
        del session_tools[session_id]
    if session_id in session_notis:
        _, notis = session_notis[session_id]
        for noti in notis.values():
            noti.stop()
        del session_notis[session_id]

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()